In [64]:
import json
import re
import os
import snowflake.connector
import pandas as pd
import dtlpy as dataloop

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
snowflake_conn = snowflake.connector.connect(
                user=os.environ['SNOW_USER'],
                password=os.environ['SNOW_PASSWORD'],
                account='cai.us-east-1',
                role='SNOWFLAKE_CA_DATA_SCIENCE_RISK',
                warehouse='WHS_CA_DATA_SCIENCE_INTERMEDIATE',
                authenticator='externalbrowser',
            )

Initiating login request with your identity provider. A browser window should have opened for you to complete the login. If you can't see it, check existing browser windows, or your OS settings. Press CTRL+C to abort and try again...
Going to open: https://coxauto.okta.com/app/snowflake/exk1h5nt7snJPsOQP0h8/sso/saml?SAMLRequest=lZLRbtowFIZfJfKuEzuhdMUCqgyGmqkrKSS92J2bHMAisVMfh9C3nwll6i5aqXeR8%2F32Z%2F9nfHusK%2B8ABqVWExIGjHigCl1KtZ2QPFv4N8RDK1QpKq1gQl4Bye10jKKuGh63dqdW8NICWs9tpJD3PyakNYprgRK5EjUgtwVfx7%2FveRQw3hhtdaEr8i7yeUIggrHO8BIpUTq9nbUNp7TruqAbBNpsacQYo2xEHXVCvl34o7vTB3xI2dWJd4TD0ze3H1Kdn%2BAzreczhPwuy1I%2FXa4z4sUX1ZlW2NZg1mAOsoB8dX8WQGcwi5OgRR8EWj8MUOluU4k9FLpuWut2DNwX3UBJK72V7tLJfEKavSzL9tA922O5yYv85%2Fbu4aUbhovHVaayJL%2FW86vrxmwga5bZCAviPV1ajU6tJogtJOrUpXVLLBr4YeSzYRaO%2BIDxKAxGg%2BgP8eauS6mE7ZMX4UIfRWt1oPdW9HKiaeg%2FbwrHfbgbKvsd1a8Ul48p291QRE1P1ZLztPBewEy%2F8gZj%2Bj75NnQProdknupKFq%2FeQpta2I9rCoOwX5Glv%2BlRDrWQVVyWBhBdXVWlu5kBYd1sW9MCodPzqf9P9%2FQv&RelayState=53713 to authentic

In [4]:
fyuse_data_file =  './fyuse_data/exterior_clean_20230401_20230301.json'

In [46]:
work_order_nums = set()
transaction_records = []

In [47]:
additional_info = ['mirrored_finish', 'reflections', 'vehicle_surface', 'light_conditions', 'weather_conditions']

In [48]:
with open(fyuse_data_file) as fyuse_file:
    json_data = json.load(fyuse_file)

In [49]:
for single_record in json_data["hits"]['hits']:
    image_date = single_record['_source']['timestamp'][0:10]
    image_work_order = single_record['_source']['work_order']
    image_car_type = single_record['_source']['car_type']

    sql_query = (
        f" SELECT "
        f"  WORK_ORDER_NUM, "
        f"  VIN, "
        f"  AUCTION_CD, "
        f"  OFFRNG_REG_DT, "
        f"  CA_VEHICLE_YEAR, "
        f"  CA_VEHICLE_MAKE, "
        f"  CA_VEHICLE_MODEL, "
        f"  CA_VEHICLE_TRIM, "
        f"  CA_VEHICLE_STYLE_ID, "
        f"  CA_VEHICLE_COLOR_EXT, "
        f"  SRC_CLEANSED_COLOR_EXT, "
        f"  CA_VEHICLE_BODY_STYLE, "
        f"  CA_VEHICLE_CHASSIS_TYPE, "
        f"  CA_VEHICLE_CLASSIFICATION, "
        f"  CONDITION_VALUE, "
        f"  CONDITION_SCORE, "
        f"  MILEAGE, " 
        f"  ADJ_NATIONAL_MMR_AT_SALE "   
        f" from dp.cads_unified_transactions.unified_transactions "
        f"  WHERE WORK_ORDER_NUM = {image_work_order} "
        f"    AND OFFRNG_REG_DT between dateadd(day, -19, to_date('{image_date}')) and "
        f"        dateadd(day, 60, to_date('{image_date}')) "
        f"  order by ABS(DATEDIFF(day, to_date('{image_date}'), OFFRNG_REG_DT)) ASC"  
    )

    cur = snowflake_conn.cursor()
    transaction_data = cur.execute(sql_query)

    transaction_df = transaction_data.fetch_pandas_all()

    for col in additional_info:
        transaction_df[col] = single_record['_source'][col]

    transaction_df['image_id'] = single_record['_id']
    transaction_df['image_date'] = image_date
    transaction_df['date_diff'] = pd.to_datetime(transaction_df['OFFRNG_REG_DT']) - pd.to_datetime(transaction_df['image_date'])

    transaction_dict = transaction_df.to_dict(orient='records')

    if image_work_order not in work_order_nums:
        try:
            for row in transaction_dict:
                transaction_records.append(row)
        except IndexError:
            pass

    work_order_nums.add(image_work_order)
    
transaction_records_df = pd.DataFrame.from_dict(transaction_records)

transaction_records_df['abs_date_diff'] = transaction_records_df['date_diff'].apply(lambda x: abs(int(x.days)))
transaction_records_sorted = transaction_records_df.sort_values(by=['WORK_ORDER_NUM', 'abs_date_diff'])
transaction_records_dedup = transaction_records_sorted.drop_duplicates(subset='WORK_ORDER_NUM', keep='first')

In [60]:
missing_work_orders = work_order_nums.difference(list(transaction_records_dedup['WORK_ORDER_NUM']))

In [66]:
dataloop.login_m2m(email=os.environ['DATALOOP_USER'], password=os.environ['DATALOOP_PASSWORD'])
proj = dataloop.projects.get("Car Condition Classification")
dataset = proj.datasets.get("all_portrait")

all_names = {}

# Get one image from each vehicle at random
pages = dataset.items.list()
for page in pages:
    for item in page:
        all_names[item.id] = item.name

all_names_swap = {v: k for k, v in all_names.items()}

Iterate Pages: 100%|██████████| 66/66 [00:48<00:00,  1.35it/s]


In [92]:
for i in range(len(transaction_records_dedup)):
    vehicle_image_id = transaction_records_dedup['image_id'].iloc[i]
    
    single_vehicle_ids = []
    for key in all_names_swap:
        if re.match(vehicle_image_id, key):
            single_vehicle_ids.append(all_names_swap[key])
    
    for image in single_vehicle_ids:
        item = dataset.items.get(item_id=image)
        item.metadata['user']  = dict()
   
        for col in list(transaction_records_dedup.columns):
            val = str(transaction_records_dedup[col][transaction_records_dedup['image_id']==vehicle_image_id].to_list()[0])
            item.metadata['user'][col] = val
        item.update() 

In [93]:
item

Item(dataset_url='https://gate.dataloop.ai/api/v1/datasets/65552b994f2e91b053fdd998', created_at='2023-11-15T20:37:04.298Z', updated_at='2023-11-15T20:38:46.587Z', updated_by='rubiks@dataloop.ai', dataset_id='65552b994f2e91b053fdd998', filename='/aMdaOYcBUMhyNhiEEWEs_9.jpg', name='aMdaOYcBUMhyNhiEEWEs_9.jpg', type='file', id='65552c56edffcc8a6907f639', spec=None, creator='ashley.lawrencehuizenga@coxautoinc.com', _description=None, annotations_count=1)